## Raw data

In [1]:
import csv
import numpy as np
import pandas as pd
import json
from datetime import datetime, timedelta
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None) # show all columns
# pd.set_option('display.max_rows', None) # show all rows
# data_ori = pd.read_csv(open('/Users/chenzewen/Desktop/Masterarbeit/DF/bc50data_.csv', 'r'), index_col=0) # ignore original index when reading csv

In [2]:
df_wet = pd.read_csv(open('C:/Users/master/Downloads/mesys_wet_data_current.csv', 'r'), index_col=0).reset_index()
df_dry = pd.read_csv(open('C:/Users/master/Downloads/mesys_dry_data_current.csv', 'r'), index_col=0).reset_index()


In [3]:
class Preprocessing:
    def __init__(self, df_bc50, df_wet, df_dry, Sensor_Abstand):
        self.df_bc50 = df_bc50
        self.df_wet = df_wet
        self.df_dry = df_dry
        self.Sensor_Abstand = Sensor_Abstand
        
    def getColumnNames(self, df):
        return list(df)
                
    def dropColumns(self, df, columns_name):
        return df.drop(columns_name, axis=1, inplace=True) # The original array is replaced when the value of inplace is true
    
    def insertColumns(self, df, pos, name, value):
        return df.insert(pos, name, value)
    
    def checkMissingValues(self, df):
        """
        Get the number of missing values detected in each row
        
        """
        
        return df.isnull().sum()
    
    def deleteInvalidTime(self, df, t_min, t_max):
        """
        t is written in format! --> %Y-%m-%d %H:%M:%S
        
        """
        for i in range(len(df)):
            if datetime.strptime(df.iloc[:,1][i], "%Y-%m-%d %H:%M:%S") >= pd.Timestamp(t_min):
                break
                
        df = df[i:]
        df_reverse = df.iloc[::-1].reset_index(drop=True) # Arrange df in reverse order and reset the index

        for j in range(len(df_reverse)):
            if datetime.strptime(df_reverse.iloc[:,1][j], "%Y-%m-%d %H:%M:%S") <= pd.Timestamp(t_max):
                break
                
        df_reverse = df_reverse[j:]
        df = df_reverse.iloc[::-1].reset_index(drop=True)
        
        """
        if t is not given
        
        
        
        """

        
        return df
    
    def tmatchCoatingStartTime(self):
        """
        Recognition of the start of coating and match in time in the table
        
        """
        
        stop = True
        
        for j in range(len(self.df_bc50)):
            if self.df_bc50['Bahngeschwindigkeit_Ist'][j] > 0.0:
                t_start = self.df_bc50['DateTime'][j] # 2020-09-09 12:00:00
                if len(np.where(self.df_wet['timestamp'] == t_start)[0]) != 0:
                    for index in np.where(self.df_wet['timestamp'] == t_start)[0]:
                        if df_wet['grammage'][index] > 0.0:
                            stop = False
                            # print(t_start)
                            break
            if not stop:
                break

        return t_start
    
    def tmatchCoatingEndTime(self):
        """
        Recognition of the end of coating and match in time in the table
        
        """
        
        stop = True
        data_reverse = self.df_bc50.iloc[::-1].reset_index(drop=True)
        
        for j in range(len(data_reverse)):
            if j != len(data_reverse) - 1:
                if data_reverse['Bahngeschwindigkeit_Ist'][j] == 0.0:
                    t_end = data_reverse['DateTime'][j]
                    if len(np.where(self.df_wet['timestamp'] == t_end)[0]) != 0:
                        for index in np.where(self.df_wet['timestamp'] == t_end)[0]:
                            if (self.df_wet['grammage'][index] != 0.0) and (self.df_wet['grammage'][index+1] == 0.0):
                                stop = False
                                # print(t_end)
                                break
            if not stop:
                break
            
        return t_end
    
    def determineFoilMass(self, df):
        
        stop = True
        foil_mass = []
        for j in range(len(df)):
            if df['Bahngeschwindigkeit_Ist'][j] == 0.0:
                t = df['DateTime'][j]
                if len(np.where(self.df_wet['timestamp'] == t)[0]) != 0:
                    for index in np.where(self.df_wet['timestamp'] == t)[0]:
                        if self.df_wet['grammage'][index] > 0.0:
                            stop = False
                            foil_mass.append(self.df_wet['grammage'][index])
                            print(t)
                            break
            if not stop:
                break
        
        return np.mean(foil_mass)
    
    def matchVirtualContinousWebStart(self, df_bc50_new):
        d1 = self.Sensor_Abstand['Mesys_Wet'] - self.Sensor_Abstand['Abwickler'] - 1/60
        d2 = self.Sensor_Abstand['Mesys_Wet'] - self.Sensor_Abstand['Abwickler'] + 1/60
        index = np.where((df_bc50_new['Integrierte x'] >= d1) & (df_bc50_new['Integrierte x'] <= d2))[0][0]
        return df_bc50_new['DateTime'][index]
               
    def matchDataParameter(self, Parameter, df_bc50_new, scaler=1, n_datapoints=500):
        n = n_datapoints
        if Parameter in ['T1_O2_Ist','T1_N2_Ist','T1_Abluft_Ist','T1_IR_Temp_Ist','T1_IR_Power_Ist,T1_Mischklappe',
                          'T2_O2_Ist','T2_N2_Ist','T2_Abluft_Ist','T2_IR_Temp_Ist','T2_IR_Power_Ist,T2_Mischklappe',
                          'T3_O2_Ist','T3_N2_Ist','T3_Abluft_Ist','T3_IR_Temp_Ist','T3_IR_Power_Ist,T3_Mischklappe']:
            
            list1 = []
            if Parameter.split("_")[0] == 'T1':
                length = self.Sensor_Abstand['T1_Ausgang'] - self.Sensor_Abstand['T1_Eingang']
                distance = self.Sensor_Abstand['T1_Eingang']
            elif Parameter.split("_")[0] == 'T2':
                length = self.Sensor_Abstand['T2_Ausgang'] - self.Sensor_Abstand['T2_Eingang']
                distance = self.Sensor_Abstand['T2_Eingang']
            elif Parameter.split("_")[0] == 'T3':
                length = self.Sensor_Abstand['T3_Ausgang'] - self.Sensor_Abstand['T3_Eingang']
                distance = self.Sensor_Abstand['T3_Eingang']
            
                
            for i in range(0,n+1):
                if i/60*scaler < distance:
                    x_T = None
                    x_T = (i/60*scaler, x_T)
                    list1.append(x_T)
                else:
                    index_T = np.where((df_bc50_new['Integrierte x'] >= 9.7655+i/60*scaler-1/60) & (df_bc50_new['Integrierte x'] <=  9.7655+i/60*scaler+1/60))[0][0]
                    list2 = []
                    for m,n in enumerate(np.arange(0, length, 1/60)):
                        x_T_ = (n, df_bc50_new[Parameter][index_T+m])
                        list2.append(x_T_)
                    x_T = (i/60*scaler, list2)
                    list1.append(x_T)
            return pd.DataFrame(list1, columns=['x-Folie', Parameter])
        
        elif Parameter in ['Abwickler11_F_Ist', 'Abwickler11_D_Ist','Aufwickler61_F_Ist','Aufwickler61_D_Ist']:
            list3 = []
            for j in range(0,n+1):
                index_T = np.where((df_bc50_new['Integrierte x'] >= 9.7655+j/60*scaler-1/60) & (df_bc50_new['Integrierte x'] <=  9.7655+j/60*scaler+1/60))[0][0]
                x_A = (j/60*scaler, df_bc50_new[Parameter][index_T])
                list3.append(x_A)
            return pd.DataFrame(list3, columns=['x-Folie', Parameter])
        
        elif Parameter in ['Kamera']:
            list_kamera = []
            distance = self.Sensor_Abstand['Kamera']
            for k in range(0,n+1):
                if k/60*scaler < distance:
                    x_T = None
                    x_T = (k/60*scaler, x_T)
                    list_kamera.append(x_T)
                else:
                    index_K = np.where((df_bc50_new['Integrierte x'] >= 9.7655+k/60*scaler-1/60) & (df_bc50_new['Integrierte x'] <=  9.7655+k/60*scaler+1/60))[0][0]
                    x_K = (k/60*scaler, df_bc50_new['DateTime'][index_K])
                    list_kamera.append(x_K)
            return pd.DataFrame(list_kamera, columns=['x-Folie', Parameter])
        
        elif Parameter in ['Mesys_Wet']:
            list_wet = []
            
            for m in range(0,n+1):
                index_W = np.where((df_bc50_new['Integrierte x'] >= 9.7655+m/60*scaler-1/60) & (df_bc50_new['Integrierte x'] <=  9.7655+m/60*scaler+1/60))[0][0]
                time = df_bc50_new['DateTime'][index_W]
                list_wet_y = []
                for t in np.where(self.df_wet['timestamp'] == time)[0]:
                    x_M_ = (df_wet['y_position'][t]/1000, df_wet['grammage'][t])
                    list_wet_y.append(x_M_)
                x_M = (m/60*scaler, list_wet_y)
                list_wet.append(x_M)
            return pd.DataFrame(list_wet, columns=['x-Folie', Parameter])
        
        else:
            raise Exception("This Parameter is not available!")
            


        

## Remove unnecessary parameters 

1. The separator of the original data is a comma!
2. Remove column starting with 'unnamed' by lower() in dataframe
3. After converting csv to dataframe, all data types become strings (solved)

In [4]:
data = pd.read_csv(open('C:/Users/master/Downloads/Vorverarbeitung.csv', 'rb'), low_memory=False, skiprows=1)
# data = pd.read_csv('/Users/chenzewen/Desktop/Masterarbeit/DF/Vorverarbeitung_SPS_Daten.csv', index_col=0, sep=';', low_memory=False)
cols = [col for col in data.columns if col.lower()[:7] != 'unnamed']
data = data[cols]
# data.insert(2, 'x-Koordinate', np.nan)

## Set the distance between sensors

In [5]:
Sensor_Abstand = {"Abwickler": -9.7655, 
                  "Corona": -6.8685, 
                  "Mesys_Wet": 0, 
                  "T1_Eingang": 0.7605, 
                  "T1_Ausgang": 1.7585, 
                  "T2_Eingang": 1.7585,
                  "T2_Ausgang": 2.7755,
                  "T3_Eingang": 3.7325,
                  "T3_Ausgang": 4.7345,
                  "Mesys_Dry": 5.3045,
                  "Kamera": 5.566,
                  "Aufwickler": 11.2425}

## Initialization

1. **Input**：Datasets for bc_50, Mesys_wet, Mesys_dry
2. Check for missing values and add integrated coordinates of x

In [6]:
p = Preprocessing(data, df_wet, df_dry, Sensor_Abstand)
p.checkMissingValues(data)
p.insertColumns(data, 2, 'Integrierte x', np.nan)

3. Determine the start time (The time the machine is running, but the coating has not started), end time and foil mass...

In [7]:
p.tmatchCoatingStartTime()

'2022-09-09 09:48:25'

In [8]:
p.tmatchCoatingEndTime()

'2022-09-09 17:38:21'

In [9]:
data_new = p.deleteInvalidTime(data, p.tmatchCoatingStartTime(), p.tmatchCoatingEndTime())


In [10]:
p.determineFoilMass(data_new)

2022-09-09 09:51:47


41.95

## Calculate the x-axis coordinates

In [11]:
import sys
for i in range(len(data_new['Integrierte x'])):
    if i == 0:
        data_new['Integrierte x'][0] = 0
    else:
        d_before = datetime.strptime(data_new['DateTime'][0], "%Y-%m-%d %H:%M:%S")
        d_after = datetime.strptime(data_new['DateTime'][1], "%Y-%m-%d %H:%M:%S")
        data_new['Integrierte x'][i] = data_new['Integrierte x'][i - 1] + data_new['Bahngeschwindigkeit_Ist'][i - 1] / 60.0 * timedelta.total_seconds(d_after - d_before)

<ipython-input-11-95087fbbba8b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new['Integrierte x'][0] = 0
<ipython-input-11-95087fbbba8b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new['Integrierte x'][i] = data_new['Integrierte x'][i - 1] + data_new['Bahngeschwindigkeit_Ist'][i - 1] / 60.0 * timedelta.total_seconds(d_after - d_before)


## Virtual continous web start time (x = 9.7655m)

In [12]:
p.matchVirtualContinousWebStart(data_new)

'2022-09-09 11:04:20'

In [13]:
data_new

,ID,DateTime,Integrierte x,Meterzaehler,Bahngeschwindigkeit_Ist,Abwickler11_F_Ist,Abwickler11_D_Ist,Zugwalze16_Korr,Zugwalze16_V_Ist,Zugwalze16_md_Ist,Beschichtung21_V_Ist,Beschichtung21_md_Ist,T1_O2_Ist,T1_N2_Ist,T1_Abluft_Ist,T1_IR_Temp_Ist,T1_IR_Power_Soll,T1_IR_Power_Ist,T1_Mischklappe,T2_O2_Ist,T2_N2_Ist,T2_Abluft_Ist,T2_IR_Temp_Ist,T2_IR_Power_Soll,T2_IR_Power_Ist,T2_Mischklappe,T3_O2_Ist,T3_N2_Ist,T3_Abluft_Ist,T3_IR_Temp_Ist,T3_IR_Power_Soll,T3_IR_Power_Ist,T3_Mischklappe,Sammel_Abluft_Ist,Aufwickler61_F_Ist,Aufwickler61_D_Ist,Zugwalze51_V_Ist,Zugwalze51_md_Ist
0,1660783,2022-09-09 09:48:25,0.000000,113.3,0.5,27.4619,287.353,-0.01,0.499376,5.740510,0.499779,4.256310,21.0604,0.113916,-0.308292,22.5524,118.8440,80.0,15,20.7062,0.14757,-0.076359,19.5007,148.3130,100.0000,35,20.5250,0.070377,0.286800,16.9067,150.127,100.0,25,0,24.3062,92.3154,0.500048,5.419560
1,1660784,2022-09-09 09:48:27,0.016667,113.4,0.5,27.6042,291.930,-0.01,0.498973,5.742350,0.500182,4.271050,21.0671,0.112917,-0.219791,22.8576,118.8440,80.0,15,20.7010,0.14757,-0.048893,19.8059,147.9650,100.0000,35,20.5240,0.072423,0.286800,17.1567,150.127,100.0,25,0,24.3511,92.2544,0.500182,5.236290
2,1660785,2022-09-09 09:48:30,0.033333,113.4,0.5,29.2546,299.377,-0.01,0.498839,6.105730,0.500182,4.323490,21.0673,0.112847,-0.158756,22.5524,118.7830,80.0,15,20.6956,0.14757,-0.048893,20.0805,148.5080,100.0000,35,20.5183,0.069445,0.259335,17.1567,150.188,100.0,25,0,24.3305,92.1628,0.500048,5.335650
3,1660786,2022-09-09 09:48:32,0.050000,113.4,0.5,27.9923,305.938,-0.01,0.499645,5.532530,0.500182,4.271950,21.0619,0.112847,-0.219791,22.5524,118.9660,80.0,15,20.7010,0.14757,-0.076359,20.0805,148.4410,100.0000,35,20.5227,0.071149,0.378351,17.2119,150.188,100.0,25,0,24.3428,92.0713,0.499913,5.254390
4,1660787,2022-09-09 09:48:34,0.066667,113.4,0.5,26.3857,311.675,-0.01,0.500048,5.587740,0.499779,4.308310,21.0591,0.112847,-0.277775,22.5524,118.9660,80.0,15,20.7051,0.14757,-0.076359,19.8059,148.3740,100.0000,35,20.5244,0.069445,0.286800,16.9067,150.188,100.0,25,0,24.9409,91.9797,0.500450,5.289000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13149,1673932,2022-09-09 17:38:13,153.725111,27.5,0.0,29.4129,178.466,-0.04,0.000000,0.038594,0.000000,0.011310,20.2898,0.112847,-5.065970,57.3424,82.6269,80.0,15,13.8963,0.15191,-5.120900,64.5750,69.8618,70.0981,35,19.7459,0.069445,-4.095510,50.1097,106.841,100.0,25,100,37.1451,109.4050,0.000269,0.031175
13150,1673933,2022-09-09 17:38:15,153.725111,27.5,0.0,29.4187,178.466,-0.04,0.000000,0.037435,0.000000,0.011976,20.2898,0.112847,-5.065970,57.3424,82.6269,80.0,15,14.0548,0.15191,-5.212450,64.5750,70.6252,71.1402,35,19.7459,0.069445,-4.187070,50.1097,106.963,100.0,25,100,37.5229,109.4050,0.000134,0.024400
13151,1673934,2022-09-09 17:38:17,153.725111,27.5,0.0,29.4213,178.466,-0.04,0.000000,0.040161,0.000000,0.010444,20.2898,0.112847,-5.065970,57.0372,82.7978,80.0,15,14.0548,0.15191,-5.212450,64.2699,71.0882,70.5124,35,19.7459,0.069445,-4.095510,50.1097,106.902,100.0,25,100,37.8922,109.4050,0.000134,0.023625
13152,1673935,2022-09-09 17:38:19,153.725111,27.5,0.0,29.4135,178.466,-0.04,0.000000,0.043621,0.000000,0.010756,20.2898,0.112847,-5.096490,56.7625,82.7978,80.0,15,14.0548,0.15191,-5.239920,63.9647,71.6115,71.1307,35,19.4474,0.069445,-4.156550,50.9947,106.017,100.0,25,100,38.2764,109.4050,0.000134,0.030836


**Output**：returns the parameter values based on the given parameter ('Mesys_Wet', 'Kamera', 'T1_O2_Ist', 'T1_N2_Ist', 'T1_Abluft_Ist', 'T1_IR_Temp_Ist', 'T1_IR_Power_Ist', 'T1_Mischklappe',
'Abwickler11_F_Ist', 'Abwickler11_D_Ist', 'Aufwickler61_F_Ist', 'Aufwickler61_D_Ist')

In [17]:
p.matchDataParameter('Abwickler11_F_Ist', data_new, 1, 500)

,x-Folie,Abwickler11_F_Ist
0,0.000000,23.1188
1,0.016667,23.3380
2,0.033333,23.6740
3,0.050000,23.1762
4,0.066667,22.9503
...,...,...
496,8.266667,24.8076
497,8.283333,24.7859
498,8.300000,25.5319
499,8.316667,25.4785


## Writing draft of code

In [28]:
scaler=1
index_W = np.where((data_new['Integrierte x'] >= 9.7655+42/60*scaler-1/60) & (data_new['Integrierte x'] <=  9.7655+42/60*scaler+1/60))[0][0]
time = data_new['DateTime'][index_W]
time

'2022-09-09 11:05:50'

In [17]:
data_new[2126:2138]

,ID,DateTime,Integrierte x,Meterzaehler,Bahngeschwindigkeit_Ist,Abwickler11_F_Ist,Abwickler11_D_Ist,Zugwalze16_Korr,Zugwalze16_V_Ist,Zugwalze16_md_Ist,Beschichtung21_V_Ist,Beschichtung21_md_Ist,T1_O2_Ist,T1_N2_Ist,T1_Abluft_Ist,T1_IR_Temp_Ist,T1_IR_Power_Soll,T1_IR_Power_Ist,T1_Mischklappe,T2_O2_Ist,T2_N2_Ist,T2_Abluft_Ist,T2_IR_Temp_Ist,T2_IR_Power_Soll,T2_IR_Power_Ist,T2_Mischklappe,T3_O2_Ist,T3_N2_Ist,T3_Abluft_Ist,T3_IR_Temp_Ist,T3_IR_Power_Soll,T3_IR_Power_Ist,T3_Mischklappe,Sammel_Abluft_Ist,Aufwickler61_F_Ist,Aufwickler61_D_Ist,Zugwalze51_V_Ist,Zugwalze51_md_Ist
2126,1662909,2022-09-09 11:04:20,9.761333,124.1,0.5,23.1188,204.101,-0.01,0.500182,6.93931,0.500450,4.54403,-0.066547,3.00781,-1.086490,51.9408,17.7248,24.98650,15,-0.058987,2.64766,-1.062070,76.5074,23.1578,23.0287,35,0.704025,2.99045,-0.604311,77.3008,35.4087,33.7600,25,50,24.2360,93.5972,0.499510,4.34816
2127,1662910,2022-09-09 11:04:22,9.778000,124.1,0.5,23.3380,202.209,-0.01,0.499913,7.00743,0.500182,4.54940,-0.053502,3.00361,-1.025450,53.1615,17.1200,17.54840,15,-0.053198,2.65191,-0.912539,76.8125,23.5251,25.4011,35,0.710302,2.99910,-0.634829,72.5401,37.4817,52.9400,25,50,24.2583,93.5667,0.499913,4.30735
2128,1662911,2022-09-09 11:04:24,9.794667,124.1,0.5,23.6740,200.530,-0.01,0.499108,7.03637,0.500048,4.52090,-0.059877,3.00347,-1.025450,69.3663,14.4898,5.57839,15,-0.062859,2.65245,-0.973574,76.2022,23.4654,24.1879,35,0.704052,2.99565,-0.634829,68.8592,40.9131,47.1089,25,50,24.1941,93.5361,0.500182,4.35007
2129,1662912,2022-09-09 11:04:27,9.811333,124.2,0.5,23.1762,199.096,-0.01,0.499242,6.90871,0.499376,4.53707,-0.062249,3.00781,-1.055970,96.9541,-26.1447,0.00000,15,-0.065069,2.64621,-0.943056,74.7068,24.5682,28.6025,35,0.704027,2.99913,-0.634829,67.4437,43.4592,50.4809,25,50,23.8313,93.4446,0.499913,4.29322
2130,1662913,2022-09-09 11:04:29,9.828000,124.2,0.5,22.9503,197.326,-0.01,0.500719,6.87358,0.500182,4.65957,-0.063802,3.00781,-1.086490,98.1443,-27.1212,0.00000,15,-0.062651,2.64315,-1.031560,75.0120,25.2221,28.6806,35,0.704965,2.98336,-0.515810,65.3685,48.3622,64.5918,25,50,23.8542,93.3530,0.500316,4.27273
2131,1662914,2022-09-09 11:04:31,9.844667,124.2,0.5,23.3779,195.831,-0.01,0.499108,7.01909,0.499913,4.56708,-0.064728,3.00781,-0.997988,96.6489,-26.3278,0.00000,15,-0.061928,2.65229,-1.031560,74.1270,25.6531,27.1000,35,0.707185,2.98177,-0.665346,68.2535,46.8844,43.7235,25,50,24.1983,93.2004,0.500719,4.12234
2132,1662915,2022-09-09 11:04:33,9.861333,124.2,0.5,23.3228,194.488,-0.01,0.499376,6.95705,0.499779,4.57778,-0.059535,3.01197,-1.025450,94.8484,-25.0155,0.00000,15,-0.062711,2.65628,-1.062070,73.5167,26.1148,27.8141,35,0.704028,2.98176,-0.634829,69.1292,43.3372,30.1425,25,50,24.9768,93.0479,0.500182,4.18748
2133,1662916,2022-09-09 11:04:35,9.878000,124.2,0.5,22.4649,193.084,-0.02,0.501122,6.82594,0.500182,4.53952,-0.066449,3.00820,-1.117000,93.6582,-23.0929,0.00000,15,-0.060316,2.65616,-1.031560,72.9368,26.7662,27.7604,35,0.706803,2.98611,-0.634829,71.4297,39.0844,22.2461,25,50,25.8775,92.9258,0.499913,4.12459
2134,1662917,2022-09-09 11:04:37,9.894667,124.3,0.5,21.2942,191.802,-0.02,0.500585,6.64678,0.499913,4.49945,-0.047686,3.00781,-1.086490,92.1628,-21.5976,0.00000,15,-0.046335,2.65629,-0.943056,75.0120,25.2711,22.0125,35,0.718951,3.00058,-0.665346,68.9390,37.8335,26.5019,25,50,26.7680,92.8953,0.499779,4.21009
2135,1662918,2022-09-09 11:04:39,9.911333,124.3,0.5,21.1983,190.673,-0.02,0.500585,6.78922,0.500182,4.58659,-0.057001,3.00781,-1.055970,90.3623,-20.2060,0.00000,15,-0.056396,2.65204,-0.943056,75.0120,24.0006,20.7921,35,0.704427,2.99045,-0.604311,69.9297,51.9928,70.3060,25,50,27.0690,92.9563,0.500048,4.17904


In [18]:
np.where(data_new['Meterzaehler'] == 0.0)

(array([ 2345, 11181, 11182, 11183], dtype=int64),)

In [18]:
data_new[2345:2346]

,ID,DateTime,Integrierte x,Meterzaehler,Bahngeschwindigkeit_Ist,Abwickler11_F_Ist,Abwickler11_D_Ist,Zugwalze16_Korr,Zugwalze16_V_Ist,Zugwalze16_md_Ist,Beschichtung21_V_Ist,Beschichtung21_md_Ist,T1_O2_Ist,T1_N2_Ist,T1_Abluft_Ist,T1_IR_Temp_Ist,T1_IR_Power_Soll,T1_IR_Power_Ist,T1_Mischklappe,T2_O2_Ist,T2_N2_Ist,T2_Abluft_Ist,T2_IR_Temp_Ist,T2_IR_Power_Soll,T2_IR_Power_Ist,T2_Mischklappe,T3_O2_Ist,T3_N2_Ist,T3_Abluft_Ist,T3_IR_Temp_Ist,T3_IR_Power_Soll,T3_IR_Power_Ist,T3_Mischklappe,Sammel_Abluft_Ist,Aufwickler61_F_Ist,Aufwickler61_D_Ist,Zugwalze51_V_Ist,Zugwalze51_md_Ist
2345,1663128,2022-09-09 11:12:09,13.411333,0.0,0.5,22.8696,201.812,-0.02,0.50045,5.57534,0.499779,4.474,-0.055525,3.00781,-1.23602,57.9527,7.13025,6.66714,15,-0.058047,2.66927,-1.03156,74.7068,11.9152,11.3931,35,0.860675,2.98867,-0.634829,87.2683,9.10955,9.56137,25,50,24.2238,93.8108,0.500182,4.42776


In [29]:
df_wet[7090:7130]

,ID,timestamp,grammage,grammage_average,x_position,x_velocity,y_position,y_velocity
7090,895909,2022-09-09 11:05:25,350.20,189.183,95804,71.2738,3095,500.0
7091,895910,2022-09-09 11:05:27,320.60,216.077,95912,57.7311,3745,500.0
7092,895911,2022-09-09 11:05:27,3.20,189.173,95962,49.1726,4045,500.0
7093,895912,2022-09-09 11:05:28,0.00,155.485,96004,87.9448,4252,0.0
7094,895913,2022-09-09 11:05:29,40.20,147.783,96070,99.9497,3910,-500.0
7095,895914,2022-09-09 11:05:29,335.75,176.269,96120,106.6730,3610,-500.0
7096,895915,2022-09-09 11:05:30,339.85,208.038,96162,69.2899,3360,-500.0
7097,895916,2022-09-09 11:05:30,346.85,215.627,96212,64.8270,3060,-500.0
7098,895917,2022-09-09 11:05:31,13.20,192.983,96287,61.1986,2610,-500.0
7099,895918,2022-09-09 11:05:32,0.00,159.482,96329,87.6537,2370,-254.0


In [20]:
df_wet.iloc[:,1]

0        2022-09-09 00:07:52
1        2022-09-09 00:30:15
2        2022-09-09 01:18:32
3        2022-09-09 01:18:52
4        2022-09-09 06:14:23
                ...         
43091    2022-09-09 23:43:02
43092    2022-09-09 23:52:21
43093    2022-09-09 23:54:23
43094    2022-09-09 23:55:09
43095    2022-09-09 23:55:27
Name: timestamp, Length: 43096, dtype: object

In [21]:
datetime.strptime(data['DateTime'][6], "%Y-%m-%d %H:%M:%S")

datetime.datetime(2022, 9, 9, 0, 0, 14)

In [22]:
datetime.strptime(data['DateTime'][0], "%Y-%m-%d %H:%M:%S") > pd.Timestamp("2022-09-09 00:00:00")

True

In [23]:
data_r = data.iloc[::-1].reset_index(drop=True)
data_r

,ID,DateTime,Integrierte x,Meterzaehler,Bahngeschwindigkeit_Ist,Abwickler11_F_Ist,Abwickler11_D_Ist,Zugwalze16_Korr,Zugwalze16_V_Ist,Zugwalze16_md_Ist,Beschichtung21_V_Ist,Beschichtung21_md_Ist,T1_O2_Ist,T1_N2_Ist,T1_Abluft_Ist,T1_IR_Temp_Ist,T1_IR_Power_Soll,T1_IR_Power_Ist,T1_Mischklappe,T2_O2_Ist,T2_N2_Ist,T2_Abluft_Ist,T2_IR_Temp_Ist,T2_IR_Power_Soll,T2_IR_Power_Ist,T2_Mischklappe,T3_O2_Ist,T3_N2_Ist,T3_Abluft_Ist,T3_IR_Temp_Ist,T3_IR_Power_Soll,T3_IR_Power_Ist,T3_Mischklappe,Sammel_Abluft_Ist,Aufwickler61_F_Ist,Aufwickler61_D_Ist,Zugwalze51_V_Ist,Zugwalze51_md_Ist
0,1674699,2022-09-09 18:05:35,NaN,41.3,0.0,32.1518,181.671,-0.04,0.0,0.030806,0.0,0.010178,21.2216,0.117187,-1.416080,43.8536,97.3832,80.0,15,20.7043,0.15191,-1.391660,46.6613,117.026,100.0,35,20.6933,0.069445,-1.080390,36.9567,121.775,100.0,25,50,21.7021,91.6441,0.0,0.006907
1,1674698,2022-09-09 18:05:33,NaN,41.3,0.0,32.1461,181.640,-0.04,0.0,0.029866,0.0,0.014017,21.2216,0.117187,-1.446590,44.1588,97.3832,80.0,15,20.7043,0.15191,-1.181090,46.6613,116.843,100.0,35,20.6932,0.069445,-1.052920,36.7067,121.781,100.0,25,50,21.6978,91.6441,0.0,0.009470
2,1674697,2022-09-09 18:05:31,NaN,41.3,0.0,32.1425,181.640,-0.04,0.0,0.026010,0.0,0.014457,21.2216,0.117187,-1.416080,44.1588,97.1391,80.0,15,20.7043,0.15191,-1.330630,46.9664,116.788,100.0,35,20.6926,0.069445,-1.052920,36.9567,122.020,100.0,25,50,21.6836,91.6746,0.0,0.008908
3,1674696,2022-09-09 18:05:28,NaN,41.3,0.0,32.1454,181.640,-0.04,0.0,0.024093,0.0,0.013611,21.2216,0.117187,-1.266540,44.1588,97.2001,80.0,15,20.7043,0.15191,-1.330630,46.9664,116.904,100.0,35,20.6905,0.069445,-1.080390,36.6515,121.897,100.0,25,50,21.6752,91.6746,0.0,0.008637
4,1674695,2022-09-09 18:05:26,NaN,41.3,0.0,32.1237,181.610,-0.04,0.0,0.027791,0.0,0.016222,21.2216,0.117187,-1.446590,44.1588,97.2001,80.0,15,20.7043,0.15191,-1.300110,47.2411,116.965,100.0,35,20.6933,0.069445,-0.872866,36.2067,121.836,100.0,25,50,21.6671,91.6746,0.0,0.010405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30348,1644351,2022-09-09 00:00:09,NaN,112.9,0.0,22.0508,187.286,-0.01,0.0,0.014672,0.0,0.006338,21.0669,0.117187,-0.100773,22.2473,118.9660,80.0,15,20.7006,0.14757,-0.076359,21.2707,147.855,100.0,35,20.6859,0.069445,0.317318,18.0722,149.114,100.0,25,0,20.0224,156.1580,0.0,0.021131
30349,1644350,2022-09-09 00:00:07,NaN,112.9,0.0,22.0459,187.286,-0.01,0.0,0.018610,0.0,0.003729,21.0663,0.117187,-0.128239,22.2473,119.0270,80.0,15,20.7006,0.14757,-0.048893,20.0805,147.916,100.0,35,20.6859,0.069458,0.347836,18.0722,149.279,100.0,25,0,20.0421,156.1580,0.0,0.017251
30350,1644349,2022-09-09 00:00:05,NaN,112.9,0.0,22.0399,187.286,-0.01,0.0,0.017469,0.0,0.006610,21.0663,0.117187,-0.100773,22.5524,119.1490,80.0,15,20.7006,0.14757,-0.048893,20.0805,147.733,100.0,35,20.6859,0.069445,0.286800,17.8469,149.114,100.0,25,0,20.0451,156.1580,0.0,0.019506
30351,1644348,2022-09-09 00:00:03,NaN,112.9,0.0,22.0362,187.286,-0.01,0.0,0.019324,0.0,0.006808,21.0663,0.117187,-0.128239,22.5524,118.9050,80.0,15,20.7006,0.14757,-0.137394,20.3857,147.794,100.0,35,20.6859,0.069445,0.317318,18.0722,149.114,100.0,25,0,20.0428,156.1580,0.0,0.022977


In [24]:
data

,ID,DateTime,Integrierte x,Meterzaehler,Bahngeschwindigkeit_Ist,Abwickler11_F_Ist,Abwickler11_D_Ist,Zugwalze16_Korr,Zugwalze16_V_Ist,Zugwalze16_md_Ist,Beschichtung21_V_Ist,Beschichtung21_md_Ist,T1_O2_Ist,T1_N2_Ist,T1_Abluft_Ist,T1_IR_Temp_Ist,T1_IR_Power_Soll,T1_IR_Power_Ist,T1_Mischklappe,T2_O2_Ist,T2_N2_Ist,T2_Abluft_Ist,T2_IR_Temp_Ist,T2_IR_Power_Soll,T2_IR_Power_Ist,T2_Mischklappe,T3_O2_Ist,T3_N2_Ist,T3_Abluft_Ist,T3_IR_Temp_Ist,T3_IR_Power_Soll,T3_IR_Power_Ist,T3_Mischklappe,Sammel_Abluft_Ist,Aufwickler61_F_Ist,Aufwickler61_D_Ist,Zugwalze51_V_Ist,Zugwalze51_md_Ist
0,1644347,2022-09-09 00:00:01,NaN,112.9,0.0,22.0357,187.286,-0.01,0.0,0.017374,0.0,0.003182,21.0692,0.117187,-0.100773,22.2473,119.0270,80.0,15,20.7006,0.14757,-0.076359,20.3857,147.794,100.0,35,20.6859,0.069445,0.347836,18.0722,149.279,100.0,25,0,20.0345,156.1580,0.0,0.034692
1,1644348,2022-09-09 00:00:03,NaN,112.9,0.0,22.0362,187.286,-0.01,0.0,0.019324,0.0,0.006808,21.0663,0.117187,-0.128239,22.5524,118.9050,80.0,15,20.7006,0.14757,-0.137394,20.3857,147.794,100.0,35,20.6859,0.069445,0.317318,18.0722,149.114,100.0,25,0,20.0428,156.1580,0.0,0.022977
2,1644349,2022-09-09 00:00:05,NaN,112.9,0.0,22.0399,187.286,-0.01,0.0,0.017469,0.0,0.006610,21.0663,0.117187,-0.100773,22.5524,119.1490,80.0,15,20.7006,0.14757,-0.048893,20.0805,147.733,100.0,35,20.6859,0.069445,0.286800,17.8469,149.114,100.0,25,0,20.0451,156.1580,0.0,0.019506
3,1644350,2022-09-09 00:00:07,NaN,112.9,0.0,22.0459,187.286,-0.01,0.0,0.018610,0.0,0.003729,21.0663,0.117187,-0.128239,22.2473,119.0270,80.0,15,20.7006,0.14757,-0.048893,20.0805,147.916,100.0,35,20.6859,0.069458,0.347836,18.0722,149.279,100.0,25,0,20.0421,156.1580,0.0,0.017251
4,1644351,2022-09-09 00:00:09,NaN,112.9,0.0,22.0508,187.286,-0.01,0.0,0.014672,0.0,0.006338,21.0669,0.117187,-0.100773,22.2473,118.9660,80.0,15,20.7006,0.14757,-0.076359,21.2707,147.855,100.0,35,20.6859,0.069445,0.317318,18.0722,149.114,100.0,25,0,20.0224,156.1580,0.0,0.021131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30348,1674695,2022-09-09 18:05:26,NaN,41.3,0.0,32.1237,181.610,-0.04,0.0,0.027791,0.0,0.016222,21.2216,0.117187,-1.446590,44.1588,97.2001,80.0,15,20.7043,0.15191,-1.300110,47.2411,116.965,100.0,35,20.6933,0.069445,-0.872866,36.2067,121.836,100.0,25,50,21.6671,91.6746,0.0,0.010405
30349,1674696,2022-09-09 18:05:28,NaN,41.3,0.0,32.1454,181.640,-0.04,0.0,0.024093,0.0,0.013611,21.2216,0.117187,-1.266540,44.1588,97.2001,80.0,15,20.7043,0.15191,-1.330630,46.9664,116.904,100.0,35,20.6905,0.069445,-1.080390,36.6515,121.897,100.0,25,50,21.6752,91.6746,0.0,0.008637
30350,1674697,2022-09-09 18:05:31,NaN,41.3,0.0,32.1425,181.640,-0.04,0.0,0.026010,0.0,0.014457,21.2216,0.117187,-1.416080,44.1588,97.1391,80.0,15,20.7043,0.15191,-1.330630,46.9664,116.788,100.0,35,20.6926,0.069445,-1.052920,36.9567,122.020,100.0,25,50,21.6836,91.6746,0.0,0.008908
30351,1674698,2022-09-09 18:05:33,NaN,41.3,0.0,32.1461,181.640,-0.04,0.0,0.029866,0.0,0.014017,21.2216,0.117187,-1.446590,44.1588,97.3832,80.0,15,20.7043,0.15191,-1.181090,46.6613,116.843,100.0,35,20.6932,0.069445,-1.052920,36.7067,121.781,100.0,25,50,21.6978,91.6441,0.0,0.009470


In [25]:
stop = True
for j in range(len(data)):

    if data['Bahngeschwindigkeit_Ist'][j] == 0.0:
        t = data['DateTime'][j]
        if len(np.where(df_wet['timestamp'] == t)[0]) != 0:
            for index in np.where(df_wet['timestamp'] == t)[0]:
                if df_wet['grammage'][index] > 0.0:
                    stop = False
                    print(t)
                    break
    if not stop:
        break
                    


KeyboardInterrupt: 

In [ ]:
data

In [26]:
np.where(data_r['DateTime'] == '2022-09-09 17:38:23')

(array([762], dtype=int64),)

In [27]:
data_r.iloc[830:880]

,ID,DateTime,Integrierte x,Meterzaehler,Bahngeschwindigkeit_Ist,Abwickler11_F_Ist,Abwickler11_D_Ist,Zugwalze16_Korr,Zugwalze16_V_Ist,Zugwalze16_md_Ist,Beschichtung21_V_Ist,Beschichtung21_md_Ist,T1_O2_Ist,T1_N2_Ist,T1_Abluft_Ist,T1_IR_Temp_Ist,T1_IR_Power_Soll,T1_IR_Power_Ist,T1_Mischklappe,T2_O2_Ist,T2_N2_Ist,T2_Abluft_Ist,T2_IR_Temp_Ist,T2_IR_Power_Soll,T2_IR_Power_Ist,T2_Mischklappe,T3_O2_Ist,T3_N2_Ist,T3_Abluft_Ist,T3_IR_Temp_Ist,T3_IR_Power_Soll,T3_IR_Power_Ist,T3_Mischklappe,Sammel_Abluft_Ist,Aufwickler61_F_Ist,Aufwickler61_D_Ist,Zugwalze51_V_Ist,Zugwalze51_md_Ist
830,1673869,2022-09-09 17:35:58,NaN,27.5,0.000000,28.9532,178.497,-0.04,0.000000,0.050257,0.000000,0.016300,19.350500,0.112847,-5.00493,61.2486,60.5497,61.5029,15,6.210920,0.151910,-5.00188,72.0213,35.6147,35.8898,35,17.896800,0.069445,-4.06805,58.9715,77.3562,78.1337,25,100,27.4576,109.405,0.000000,0.007339
831,1673868,2022-09-09 17:35:56,NaN,27.5,0.000000,28.9066,178.497,-0.04,0.000000,0.048911,0.000000,0.014335,19.047300,0.112847,-5.09649,61.8590,59.8584,60.3413,15,6.210920,0.151910,-5.12090,72.0213,35.1005,35.0532,35,17.899300,0.069445,-4.12603,59.6371,75.8268,76.3825,25,100,27.3657,109.405,0.000000,0.004102
832,1673867,2022-09-09 17:35:53,NaN,27.5,0.000000,28.8862,178.497,-0.04,0.000000,0.051720,0.000000,0.014105,19.047300,0.112847,-4.94695,61.8590,58.9285,58.1637,15,6.210920,0.151910,-5.00188,72.0213,34.7093,35.4921,35,17.897500,0.069445,-4.12603,59.9669,74.9767,73.9528,25,100,27.2210,109.405,0.000000,0.007194
833,1673866,2022-09-09 17:35:51,NaN,27.5,0.000000,28.8685,178.497,-0.04,0.000000,0.045361,0.000000,0.014582,19.047300,0.112847,-5.00493,62.1641,58.5693,59.6245,15,6.210920,0.151910,-5.21245,72.3265,34.0261,33.5405,35,17.899300,0.069445,-4.12603,59.4117,74.4735,75.4076,25,100,27.1206,109.405,0.000000,0.007246
834,1673865,2022-09-09 17:35:49,NaN,27.5,0.000000,28.8595,178.497,-0.04,0.000000,0.051027,0.000000,0.014850,19.047300,0.112847,-5.00493,62.7440,57.9580,58.3654,15,6.210920,0.151910,-4.94390,72.3265,33.8964,34.6704,35,17.593300,0.069445,-4.09551,60.4669,72.7770,73.1649,25,100,27.0416,109.405,0.000000,0.006921
835,1673864,2022-09-09 17:35:47,NaN,27.5,0.000000,28.8543,178.497,-0.04,0.000000,0.052339,0.000000,0.013784,19.047300,0.112847,-5.00493,62.1641,57.4748,59.1977,15,6.035530,0.151910,-5.00188,72.9368,33.1026,33.3309,35,17.593300,0.069445,-4.12603,60.5772,71.7664,74.2574,25,100,26.9456,109.405,0.000000,0.004798
836,1673863,2022-09-09 17:35:45,NaN,27.5,0.000000,28.8492,178.497,-0.04,0.000000,0.046845,0.000000,0.014639,19.047300,0.112847,-4.97442,62.4388,56.5322,57.4448,15,5.444230,0.151910,-5.12090,73.2115,32.8139,33.3134,35,17.593300,0.069445,-4.15655,61.2122,70.7455,70.8726,25,100,26.8773,109.405,0.000000,0.003634
837,1673862,2022-09-09 17:35:43,NaN,27.5,0.000000,28.8439,178.497,-0.04,0.000000,0.046192,0.000000,0.012429,18.736700,0.112847,-5.06597,62.7440,55.7830,55.2416,15,5.444230,0.151910,-5.12090,73.2115,32.2413,32.4878,35,17.593300,0.069445,-4.12603,61.4622,69.3562,68.8910,25,100,26.8261,109.405,0.000000,0.001848
838,1673861,2022-09-09 17:35:41,NaN,27.5,0.000000,28.8277,178.497,-0.04,0.000000,0.038945,0.000000,0.015388,18.736700,0.112847,-5.06597,62.7440,55.5593,55.7072,15,5.444230,0.151910,-5.18194,73.5167,31.6560,31.6745,35,17.589700,0.069445,-4.12603,61.4622,69.0518,70.7820,25,100,26.8035,109.405,0.000000,0.002322
839,1673860,2022-09-09 17:35:38,NaN,27.5,0.000000,28.8140,178.497,-0.04,0.000000,0.049869,0.000000,0.014698,18.736700,0.112847,-5.06597,62.7440,55.0900,55.8238,15,5.444230,0.151910,-5.21245,73.2115,31.5665,31.9208,35,17.591800,0.069445,-4.09551,62.0726,67.8403,68.3199,25,100,26.6840,109.405,0.000000,0.002091


In [28]:
df_wet[6900:7000]

,ID,timestamp,grammage,grammage_average,x_position,x_velocity,y_position,y_velocity
6900,895719,2022-09-09 11:03:22,349.40,216.375,85493,92.9336,3253,-500.0
6901,895720,2022-09-09 11:03:23,332.00,217.983,85543,94.8844,2953,-500.0
6902,895721,2022-09-09 11:03:24,2.15,187.611,85611,104.8590,2553,-500.0
6903,895722,2022-09-09 11:03:24,0.00,150.779,85661,110.2980,2382,324.0
6904,895723,2022-09-09 11:03:25,125.35,151.533,85728,82.5331,2777,500.0
...,...,...,...,...,...,...,...,...
6995,895814,2022-09-09 11:04:24,0.25,171.986,90631,101.4430,4143,500.0
6996,895815,2022-09-09 11:04:24,0.05,144.849,90682,90.8770,4165,-500.0
6997,895816,2022-09-09 11:04:25,310.85,152.872,90749,88.1028,3765,-500.0
6998,895817,2022-09-09 11:04:26,345.35,192.400,90799,76.5531,3465,-500.0


In [29]:
p_wert = data_new['T1_O2_Ist'][np.where(data_new['Integrierte x'] >= 9.7655)[0][0]]
p_wert

-0.0535021

In [30]:
p1_wert = data_new['T1_O2_Ist'][np.where(data_new['Integrierte x'] >= 9.7655 + 0.02)[0][0]]
p1_wert

-0.0598773

In [52]:
dfghj = [(0, 20), (0.01, 23), (0.02, 25), (0.03, 26), (0.04, 27), (0.05, 28), (0.06, 29)]

In [53]:
ddd = pd.DataFrame([[0, dfghj], [0.02, p1_wert]],columns=['x-Folie', 'Parameterwert'])
ddd

,x-Folie,Parameterwert
0,0.00,"[(0, 20), (0.01, 23), (0.02, 25), (0.03, 26), ..."
1,0.02,-0.0598773


In [57]:
T_values = [T for x, T in dfghj]
np.mean(T_values)

25.428571428571427

In [18]:
list(data_new)

['ID',
 'DateTime',
 'Integrierte x',
 'Meterzaehler',
 'Bahngeschwindigkeit_Ist',
 'Abwickler11_F_Ist',
 'Abwickler11_D_Ist',
 'Zugwalze16_Korr',
 'Zugwalze16_V_Ist',
 'Zugwalze16_md_Ist',
 'Beschichtung21_V_Ist',
 'Beschichtung21_md_Ist',
 'T1_O2_Ist',
 'T1_N2_Ist',
 'T1_Abluft_Ist',
 'T1_IR_Temp_Ist',
 'T1_IR_Power_Soll',
 'T1_IR_Power_Ist',
 'T1_Mischklappe',
 'T2_O2_Ist',
 'T2_N2_Ist',
 'T2_Abluft_Ist',
 'T2_IR_Temp_Ist',
 'T2_IR_Power_Soll',
 'T2_IR_Power_Ist',
 'T2_Mischklappe',
 'T3_O2_Ist',
 'T3_N2_Ist',
 'T3_Abluft_Ist',
 'T3_IR_Temp_Ist',
 'T3_IR_Power_Soll',
 'T3_IR_Power_Ist',
 'T3_Mischklappe',
 'Sammel_Abluft_Ist',
 'Aufwickler61_F_Ist',
 'Aufwickler61_D_Ist',
 'Zugwalze51_V_Ist',
 'Zugwalze51_md_Ist']

In [18]:
"""
list1 = []
n = 500
for i in range(0,n+1):
    if i/60*1 <= 0.7605:
        x_T = None
    else:
        x_T = (i/60*1, data_new['Abwickler'][2126+i*1])
        list1.append(x_T)
"""

list1 = []

L = 0.998
n = 500
for i in range(0,n+1):
    if i/60*1 <= 0.7605:
        x_T = None
        x_T = (i/60*1, x_T)
        list1.append(x_T)
    else:
        index_T = np.where((data_new['Integrierte x'] >= 9.7655+i/60-1/60) & (data_new['Integrierte x'] <=  9.7655+i/60+1/60))[0][0]
        list2 = []
        for m,n in enumerate(np.arange(0,L, 1/60)):
            x_T_ = (n, data_new['T2_O2_Ist'][index_T+m])
            list2.append(x_T_)
        x_T = (i/60*1, list2)
        list1.append(x_T)


In [19]:
list1

[(0.0, None),
 (0.016666666666666666, None),
 (0.03333333333333333, None),
 (0.05, None),
 (0.06666666666666667, None),
 (0.08333333333333333, None),
 (0.1, None),
 (0.11666666666666667, None),
 (0.13333333333333333, None),
 (0.15, None),
 (0.16666666666666666, None),
 (0.18333333333333332, None),
 (0.2, None),
 (0.21666666666666667, None),
 (0.23333333333333334, None),
 (0.25, None),
 (0.26666666666666666, None),
 (0.2833333333333333, None),
 (0.3, None),
 (0.31666666666666665, None),
 (0.3333333333333333, None),
 (0.35, None),
 (0.36666666666666664, None),
 (0.38333333333333336, None),
 (0.4, None),
 (0.4166666666666667, None),
 (0.43333333333333335, None),
 (0.45, None),
 (0.4666666666666667, None),
 (0.48333333333333334, None),
 (0.5, None),
 (0.5166666666666667, None),
 (0.5333333333333333, None),
 (0.55, None),
 (0.5666666666666667, None),
 (0.5833333333333334, None),
 (0.6, None),
 (0.6166666666666667, None),
 (0.6333333333333333, None),
 (0.65, None),
 (0.6666666666666666, None

In [68]:
dddggg = pd.DataFrame(list1, columns=['x-Folie', 'Parameterwert'])
dddggg

,x-Folie,Parameterwert
0,0.000000,None
1,0.016667,None
2,0.033333,None
3,0.050000,None
4,0.066667,None
...,...,...
496,8.266667,"[(0.0, -0.049799), (0.016666666666666666, -0.0..."
497,8.283333,"[(0.0, -0.05780319999999999), (0.0166666666666..."
498,8.300000,"[(0.0, -0.05705980000000001), (0.0166666666666..."
499,8.316667,"[(0.0, -0.0401881), (0.016666666666666666, -0...."


In [69]:
dddggg[46:52]

,x-Folie,Parameterwert
46,0.766667,"[(0.0, -0.059686699999999995), (0.016666666666..."
47,0.783333,"[(0.0, -0.059124300000000005), (0.016666666666..."
48,0.800000,"[(0.0, -0.06676950000000001), (0.0166666666666..."
49,0.816667,"[(0.0, -0.0616375), (0.016666666666666666, -0...."
50,0.833333,"[(0.0, -0.0686349), (0.016666666666666666, -0...."
51,0.850000,"[(0.0, -0.0647512), (0.016666666666666666, -0...."


In [25]:
data_new['T1_O2_Ist'][2126]

-0.06654689999999999

In [61]:
data_new[2168:2200]

,ID,DateTime,Integrierte x,Meterzaehler,Bahngeschwindigkeit_Ist,Abwickler11_F_Ist,Abwickler11_D_Ist,Zugwalze16_Korr,Zugwalze16_V_Ist,Zugwalze16_md_Ist,Beschichtung21_V_Ist,Beschichtung21_md_Ist,T1_O2_Ist,T1_N2_Ist,T1_Abluft_Ist,T1_IR_Temp_Ist,T1_IR_Power_Soll,T1_IR_Power_Ist,T1_Mischklappe,T2_O2_Ist,T2_N2_Ist,T2_Abluft_Ist,T2_IR_Temp_Ist,T2_IR_Power_Soll,T2_IR_Power_Ist,T2_Mischklappe,T3_O2_Ist,T3_N2_Ist,T3_Abluft_Ist,T3_IR_Temp_Ist,T3_IR_Power_Soll,T3_IR_Power_Ist,T3_Mischklappe,Sammel_Abluft_Ist,Aufwickler61_F_Ist,Aufwickler61_D_Ist,Zugwalze51_V_Ist,Zugwalze51_md_Ist
2168,1662951,2022-09-09 11:05:50,10.461333,124.9,0.5,28.5711,210.723,-0.02,0.500048,6.97758,0.499645,4.54629,-0.051178,3.00781,-1.055970,77.7586,-6.76607,0.0,15,-0.053533,2.65625,-0.973574,83.0686,14.448900,7.050960,35,0.861792,2.99640,-0.634829,84.0030,44.5979,47.29450,25,50,26.2972,93.2004,0.500987,4.11282
2169,1662952,2022-09-09 11:05:52,10.478000,124.9,0.5,28.3250,211.578,-0.02,0.499913,7.03403,0.500048,4.53420,-0.062813,3.00781,-1.055970,77.4534,-6.52193,0.0,15,-0.059099,2.65626,-1.004090,83.6790,13.400600,11.653100,35,0.861943,2.99668,-0.634829,89.8740,40.8158,30.71920,25,50,26.4976,93.2004,0.500182,4.21931
2170,1662953,2022-09-09 11:05:54,10.494667,124.9,0.5,27.1093,212.310,-0.02,0.499779,6.95463,0.500316,4.53132,-0.060231,3.00781,-1.055970,76.2632,-6.21676,0.0,15,-0.059112,2.65738,-1.004090,83.6790,12.049100,11.559100,35,0.863118,2.99045,-0.634829,85.2776,42.2705,48.98910,25,50,26.4706,93.3225,0.500316,4.24191
2171,1662954,2022-09-09 11:05:56,10.511333,124.9,0.5,27.7655,212.951,-0.02,0.498302,7.39335,0.499913,4.55053,-0.069992,3.00781,-1.025450,76.8431,-6.27779,0.0,15,-0.063345,2.65222,-0.973574,83.9536,11.496800,11.375900,35,0.851613,2.99045,-0.604311,75.6106,50.8901,68.72180,25,50,26.0175,93.4751,0.499108,4.23266
2172,1662955,2022-09-09 11:05:59,10.528000,125.0,0.5,28.8909,213.287,-0.02,0.499913,7.18879,0.500182,4.65933,-0.059687,3.00781,-1.025450,76.5684,-6.05196,0.0,15,-0.064837,2.64658,-1.004090,85.4490,10.833400,6.685860,35,0.855374,2.99045,-0.634829,85.6931,39.3897,19.75470,25,50,24.9879,93.6277,0.500316,4.20312
2173,1662956,2022-09-09 11:06:01,10.544667,125.0,0.5,27.5233,213.531,-0.02,0.499913,6.92324,0.499376,4.61890,-0.059124,3.00347,-1.086490,75.9581,-5.51485,0.0,15,-0.060367,2.64316,-0.973574,86.0593,9.951370,7.734560,35,0.864202,2.98611,-0.573793,83.2869,42.3580,39.15240,25,50,24.2539,93.6887,0.499913,4.23726
2174,1662957,2022-09-09 11:06:03,10.561333,125.0,0.5,26.1698,213.745,-0.02,0.498570,6.94103,0.499779,4.60785,-0.066770,3.00781,-1.055970,75.9581,-5.27071,0.0,15,-0.058255,2.65111,-0.912539,85.4490,10.492600,12.977100,35,0.862162,2.98605,-0.543276,88.6839,44.6447,31.70140,25,50,23.7875,93.7192,0.499779,4.18189
2175,1662958,2022-09-09 11:06:05,10.578000,125.0,0.5,25.7867,213.897,-0.02,0.501122,6.86876,0.499779,4.65758,-0.061637,3.00781,-1.086490,75.6529,-5.08761,0.0,15,-0.059811,2.66899,-1.031560,87.5547,7.106160,1.185740,35,0.855556,2.99768,-0.634829,80.4019,46.4871,55.98850,25,50,23.6428,93.6887,0.499913,4.27353
2176,1662959,2022-09-09 11:06:07,10.594667,125.0,0.5,25.0225,213.836,-0.02,0.500450,6.89437,0.499913,4.54739,-0.068635,3.00368,-1.055970,76.2632,-5.14864,0.0,15,-0.069341,2.67350,-1.092590,87.2495,8.932970,13.326000,35,0.848418,3.00781,-0.634829,80.7822,49.4217,45.56390,25,50,23.6914,93.6277,0.499645,4.19254
2177,1662960,2022-09-09 11:06:09,10.611333,125.0,0.5,24.7713,213.775,-0.02,0.500182,6.89098,0.499779,4.57917,-0.064751,3.00781,-1.086490,75.6529,-4.96554,0.0,15,-0.060445,2.65191,-1.004090,88.4397,6.607390,4.117750,35,0.860258,2.99045,-0.634829,70.5447,55.9030,69.91820,25,50,24.1310,93.5361,0.499779,4.25453


In [72]:
df_dry[223:226]

,ID,timestamp,grammage,grammage_average,x_position,x_velocity,y_position,y_velocity
223,889073,2022-09-09 09:50:05,41.45,31.3757,12120,86.3782,3804,-500.0
224,889074,2022-09-09 09:50:06,0.10,27.4610,12187,93.6905,3411,-296.0
225,889075,2022-09-09 09:50:07,0.20,23.2531,12229,100.5270,3538,500.0


In [50]:
text = 'T4_O2_Ist'

In [51]:
 if text in ['T1_O2_Ist','T1_N2_Ist','T1_Abluft_Ist','T1_IR_Temp_Ist','T1_IR_Power_Ist,T1_Mischklappe',
             'T2_O2_Ist','T2_N2_Ist','T2_Abluft_Ist','T2_IR_Temp_Ist','T2_IR_Power_Ist,T2_Mischklappe',
             'T3_O2_Ist','T3_N2_Ist','T3_Abluft_Ist','T3_IR_Temp_Ist','T3_IR_Power_Ist,T3_Mischklappe']:
    print(1)